In [26]:
pip install cryptography

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from dotenv import load_dotenv
import os
import pandas as pd
import base64
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from cryptography.fernet import Fernet
from io import StringIO

In [28]:
# Load .env variables

with open('secret.key', 'rb') as f:
    key = f.read()

fernet = Fernet(key)

# decoding env.enc
with open('.env.enc', 'rb') as enc_file:
    encrypted = enc_file.read()
    decrypted = fernet.decrypt(encrypted).decode()

load_dotenv(stream=StringIO(decrypted))

True

In [29]:
import os
from cryptography.fernet import Fernet
from dotenv import load_dotenv

# decrepting env.enc
def load_encrypted_env(enc_file_path=".env.enc", key_file_path="secret.key"):
    with open(key_file_path, "rb") as key_file:
        key = key_file.read()
    fernet = Fernet(key)

    with open(enc_file_path, "rb") as enc_file:
        encrypted_data = enc_file.read()
        decrypted_data = fernet.decrypt(encrypted_data)
    with open(".env", "wb") as env_file:
        env_file.write(decrypted_data)

    load_dotenv()  

load_encrypted_env()

In [30]:
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
import os

print("Snowflake credentials from .env:")
print("User:", os.getenv("SNOWFLAKE_USER"))
print("Password:", os.getenv("SNOWFLAKE_PASSWORD"))
print("Account:", os.getenv("SNOWFLAKE_ACCOUNT"))
print("Role:", os.getenv("SNOWFLAKE_ROLE"))
print("Warehouse:", os.getenv("SNOWFLAKE_WAREHOUSE"))
print("Database:", os.getenv("SNOWFLAKE_DATABASE"))
print("Schema:", os.getenv("SNOWFLAKE_SCHEMA"))


Snowflake credentials from .env:
User: AHMEDELSAYADD
Password: RmFLSnBmZDZWY0pLSkdD          # this password is encrypted
Account: PCPBABX-KN96138
Role: ACCOUNTADMIN
Warehouse: COMPUTE_WH
Database: SALES
Schema: PUBLIC


In [33]:
import snowflake.connector
import os
import base64

def fix_base64_padding(encoded_str):
    return encoded_str + '=' * (-len(encoded_str) % 4)

try:
    encoded_password = "FaKJpfd6VcJKJGC"
    
    if not encoded_password:
        raise ValueError("SNOWFLAKE_PASSWORD environment variable not set.")
    
    encoded_password_fixed = fix_base64_padding(encoded_password)
    
    decoded_password = "FaKJpfd6VcJKJGC"

    conn = snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        password=decoded_password,
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        database=os.getenv("SNOWFLAKE_DATABASE"),
        schema=os.getenv("SNOWFLAKE_SCHEMA")
    )

    cs = conn.cursor()
    cs.execute("SELECT CURRENT_VERSION()")
    one_row = cs.fetchone()
    print(f"Connected to Snowflake. Version: {one_row[0]}")

except Exception as e:
    print(f"Error connecting to Snowflake: {e}")


Connected to Snowflake. Version: 9.21.0


In [34]:
df_customer = pd.read_csv("star_schema_tables/dim_customers.csv")
df_product = pd.read_csv("star_schema_tables/dim_products.csv")
df_time = pd.read_csv("star_schema_tables/dim_time.csv")
df_sales = pd.read_csv("star_schema_tables/fact_sales.csv")

In [ ]:
base_path = 'star_schema_tables'
dim_customers = pd.read_csv(os.path.join(base_path, 'dim_customers.csv'))
dim_products = pd.read_csv(os.path.join(base_path, 'dim_products.csv'))
dim_time = pd.read_csv(os.path.join(base_path, 'dim_time.csv'))
fact_sales = pd.read_csv(os.path.join(base_path, 'fact_sales.csv'))


from snowflake.connector.pandas_tools import write_pandas

def upload_to_snowflake(df, table_name):
    df.columns = [col.upper() for col in df.columns]
    success, nchunks, nrows, _ = write_pandas(conn, df, table_name.upper())
    print(f"Uploaded {nrows} rows to {table_name.upper()}")

upload_to_snowflake(dim_customers, 'dim_customer')
upload_to_snowflake(dim_products, 'dim_product')
upload_to_snowflake(dim_time, 'dim_time')
upload_to_snowflake(fact_sales, 'fact_sales')

# Close the connection
conn.close()